In [1]:
import torch
import warnings

# warnings.filterwarnings('ignore')
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA device: {torch.cuda.get_device_name(0)}")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

CUDA available: True
CUDA device: NVIDIA GeForce RTX 3050 Ti Laptop GPU


In [2]:
# ! pip install ipywidgets
# ! jupyter nbextension enable --py widgetsnbextension

In [3]:
from scripts.parsers import parse_sequences as parse_sequence_info

file_path = 'gait3d\\ListOfSequences.txt'
sequences = parse_sequence_info(file_path)

In [4]:
import json

selected_names_file = "./datasets/mediapipe/selected_joint_names.json"
input_data_file = "./datasets/mediapipe/dataset_v2.json"
output_data_file = "./datasets/mocap/dataset_v2.json"
triang_data_file = "./datasets/mediapipe/triangulation.json"

with open(input_data_file, 'r') as file:
    raw_input = json.load(file)

with open(output_data_file, 'r') as file:
    raw_output = json.load(file)

with open(triang_data_file, 'r') as file:
    triangulation_data = json.load(file)


In [5]:
with open(selected_names_file, 'r') as file:
    selected_names = json.load(file)

selected_names.pop('15')
selected_names.pop('16')
selected_names.pop('13')
selected_names.pop('14')
selected_names

{'27': 'lfoot',
 '28': 'rfoot',
 '25': 'ltibia',
 '26': 'rtibia',
 '23': 'lfemur',
 '24': 'rfemur',
 '11': 'lhumerus',
 '12': 'rhumerus'}

In [6]:
from utils.torch_train_utils import get_train_valid_test_set, MoCapInputDataset
from torch.utils.data import DataLoader

train_seq_set, valid_seq_set, test_seq_set = get_train_valid_test_set(sequences, 42)

print(f"train: {train_seq_set}")
print(f"valid: {valid_seq_set}")
print(f"test: {test_seq_set}")

train_ds = MoCapInputDataset(train_seq_set, sequences, selected_names, raw_input, raw_output)
valid_ds = MoCapInputDataset(valid_seq_set, sequences, selected_names, raw_input, raw_output)
test_ds = MoCapInputDataset(test_seq_set, sequences, selected_names, raw_input, raw_output)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader = DataLoader(valid_ds, batch_size=32, shuffle=False)
test_loader = DataLoader(test_ds, batch_size=32, shuffle=False)

train: ['p13s1', 'p13s3', 'p1s1', 'p1s3', 'p14s1', 'p14s3', 'p28s1', 'p28s3', 'p5s1', 'p5s3', 'p24s1', 'p24s3', 'p31s1', 'p31s3', 'p22s1', 'p22s3', 'p23s1', 'p23s3', 'p15s1', 'p15s3', 'p4s1', 'p4s3', 'p19s1', 'p19s3', 'p30s1', 'p30s3', 'p16s1', 'p16s3', 'p18s1', 'p18s3', 'p32s1', 'p32s3', 'p26s1', 'p26s3', 'p3s1', 'p3s3', 'p17s1', 'p17s3', 'p29s1', 'p29s3', 'p9s1', 'p9s3', 'p27s1', 'p27s3', 'p20s1', 'p20s3', 'p6s1', 'p6s3', 'p10s1', 'p10s3', 'p12s1', 'p12s3', 'p30s5', 'p30s7', 'p31s5', 'p31s7']
valid: ['p11s1', 'p11s3', 'p25s1', 'p25s3', 'p7s1', 'p7s3', 'p26s5', 'p26s7', 'p28s5', 'p28s7']
test: ['p8s1', 'p8s3', 'p21s1', 'p21s3', 'p2s1', 'p2s3', 'p27s5', 'p27s7', 'p29s5', 'p29s7']


In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class CustomNetForOptuna(nn.Module):
    def __init__(self, dropout=0.2, activation_name="relu"):
        super(CustomNetForOptuna, self).__init__()
        
        # to try different activation functions
        activations = {
            "relu": nn.ReLU(),
            "leaky_relu": nn.LeakyReLU(),
            "gelu": nn.GELU(),
        }
        self.activation = activations[activation_name]

        self.conv1d1 = [
            nn.Conv1d(in_channels=2, out_channels=8, kernel_size=2, padding=1) for _ in range(4) 
        ]
        self.conv1d2 = [
            nn.Conv1d(in_channels=8, out_channels=1, kernel_size=2) for _ in range(4)
        ]
        
        self.bn1 = nn.BatchNorm1d(32)
        self.fc1 = nn.Linear(32, 48)
        self.dropout1 = nn.Dropout(p=dropout)
        self.bn2 = nn.BatchNorm1d(48)
        self.fc2 = nn.Linear(48, 32)
        self.dropout2 = nn.Dropout(p=dropout)
        self.bn3 = nn.BatchNorm1d(32)
        self.fc3 = nn.Linear(32, 24)

    def forward(self, x):
        conv_outs = []
        for i, xi in enumerate(x):
            xi = xi.permute(0, 2, 1)
            conv = self.conv1d1[i](xi)
            conv = self.conv1d2[i](conv)
            conv = conv.squeeze(1)
            conv_outs.append(conv)

        concat = torch.cat(conv_outs, dim=1)
        out = self.bn1(concat)
        out = self.activation(self.bn2(self.fc1(out)))
        out = self.dropout1(out)
        out = self.activation(self.bn3(self.fc2(out)))
        out = self.dropout2(out)
        out = self.fc3(out)
        return out.view(-1, 8, 3)


In [14]:
import optuna
from torch.optim import AdamW

from utils.torch_train_utils import MPJPE


def objective(trial):
    lr = trial.suggest_float("lr", 1e-5, 1e-2, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-2, log=True)
    dropout = trial.suggest_float("dropout", 0.0, 0.5)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64, 128])
    activation = trial.suggest_categorical("activation", ["relu", "leaky_relu", "gelu"])

    train_loader =  DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(valid_ds, batch_size=batch_size)

    model = CustomNetForOptuna(dropout=dropout, activation_name=activation)
    criterion = MPJPE()
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)

    for epoch in range(20):
        model.train()
        for inputs, targets in train_loader:
            inputs = [x for x in inputs]
            targets = targets

            outputs = model(inputs)
            loss = criterion(outputs, targets)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    model.eval()
    val_loss = 0
    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs = [x for x in inputs]
            targets = targets
            preds = model(inputs)
            val_loss += criterion(preds, targets).item()

    return val_loss / len(val_loader)


In [15]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)

print("Best trial:", study.best_trial)
print("Best params:", study.best_params)

[I 2025-06-11 00:12:19,982] A new study created in memory with name: no-name-0c882922-6062-451e-ae71-cbde05c75528
[I 2025-06-11 00:12:27,285] Trial 0 finished with value: 519.2392944335937 and parameters: {'lr': 0.005671031230736875, 'weight_decay': 0.0004168259711955452, 'dropout': 0.354801880574658, 'batch_size': 128, 'activation': 'relu'}. Best is trial 0 with value: 519.2392944335937.
[I 2025-06-11 00:13:01,164] Trial 1 finished with value: 285.81169204711915 and parameters: {'lr': 0.0027999724229229063, 'weight_decay': 0.001801357679958648, 'dropout': 0.2756488153051272, 'batch_size': 16, 'activation': 'relu'}. Best is trial 1 with value: 285.81169204711915.
[I 2025-06-11 00:13:19,886] Trial 2 finished with value: 1681.4529998779296 and parameters: {'lr': 3.090879261047814e-05, 'weight_decay': 0.0016022937029121318, 'dropout': 0.38444282116442574, 'batch_size': 32, 'activation': 'leaky_relu'}. Best is trial 1 with value: 285.81169204711915.
[I 2025-06-11 00:13:27,678] Trial 3 fini

Best trial: FrozenTrial(number=27, state=TrialState.COMPLETE, values=[217.38408660888672], datetime_start=datetime.datetime(2025, 6, 11, 0, 18, 58, 22171), datetime_complete=datetime.datetime(2025, 6, 11, 0, 19, 9, 572686), params={'lr': 0.005589010994074508, 'weight_decay': 1.1906353862455155e-05, 'dropout': 0.2214033785244307, 'batch_size': 64, 'activation': 'gelu'}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'lr': FloatDistribution(high=0.01, log=True, low=1e-05, step=None), 'weight_decay': FloatDistribution(high=0.01, log=True, low=1e-06, step=None), 'dropout': FloatDistribution(high=0.5, log=False, low=0.0, step=None), 'batch_size': CategoricalDistribution(choices=(16, 32, 64, 128)), 'activation': CategoricalDistribution(choices=('relu', 'leaky_relu', 'gelu'))}, trial_id=27, value=None)
Best params: {'lr': 0.005589010994074508, 'weight_decay': 1.1906353862455155e-05, 'dropout': 0.2214033785244307, 'batch_size': 64, 'activation': 'gelu'}


In [ ]:
# Best params: {'lr': 0.005589010994074508, 'weight_decay': 1.1906353862455155e-05, 'dropout': 0.2214033785244307, 'batch_size': 64, 'activation': 'gelu'}